<a href="https://colab.research.google.com/github/aluposto/video_codec_test_11/blob/main/dcvc_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, os, platform
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi
!python --version

CUDA available: True
Tue Sep  2 15:55:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [2]:
!rm -rf DCVC
!git clone https://github.com/microsoft/DCVC.git
%cd DCVC

# Install build tools
!sudo apt-get update -y
!sudo apt-get install -y cmake g++ ninja-build

# Install PyTorch 2.6 with CUDA 12.6 (fallback to CUDA 12.1 if needed)
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 || \
 pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Python dependencies
!pip install -r requirements.txt

# Build C++ bitstream writer
%cd src/cpp
!pip install .
# Build CUDA fused inference kernels
%cd ../layers/extensions/inference
!pip install .

# Back to root
%cd ../../../..

Cloning into 'DCVC'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 453 (delta 75), reused 44 (delta 44), pack-reused 326 (from 2)
Receiving objects: 100% (453/453), 7.43 MiB | 13.59 MiB/s, done.
Resolving deltas: 100% (184/184), done.
/content/DCVC
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cloud.r-project.

In [3]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Torch: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4


In [4]:
!pip install tqdm requests


In [5]:
import requests
from tqdm import tqdm

url = "https://ultravideo.fi/video/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"

# Ensure folder exists
import os
os.makedirs("/media/data/UVG", exist_ok=True)

# Stream download with progress bar
response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)

Downloading: 100%|██████████| 925M/925M [00:40<00:00, 22.6MB/s]

✅ Download finished: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z


In [6]:
!sudo apt-get install -y p7zip-full
!7z x /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/UVG/
!ls -lh /media/data/UVG

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /media/data/UVG/                          1 file, 925430047 bytes (883 MiB)

Extracting archive: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
--
Path = /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
Type = 7z
Physical Size = 925430047
Headers Size = 246
Method = LZMA:16 BZip2
Solid = -
Blocks = 2

  0%      0% - Beauty_1920x1080_120fps_420_8bit_YUV.yuv                                               

In [7]:
!ls -lh /media/data/UVG


total 2.6G
-rw-r--r-- 1 root root 883M Sep  2 16:01 Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Beauty_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 Beauty_Copyright.txt


In [8]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 2 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --save_decoded_frame 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1

/content/DCVC
testing 2 rates, using qp: 0, 63, 
  0% 0/106 [00:00<?, ?it/s]created folder: out_bin/UVG
encoding/decoding 600 frames, average encoding time 44.751 ms, average decoding time 52.148 ms.
  1% 1/106 [02:48<4:54:34, 168.33s/it]encoding/decoding 600 frames, average encoding time 45.619 ms, average decoding time 53.295 ms.
  2% 2/106 [05:29<4:45:43, 164.84s/it]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with_stream(p_frame_net, i_frame_net, args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 147, in run_one_point_with_stream
    src_reader = get_src_reader(args)
                 ^^^^^

In [9]:
import requests
from tqdm import tqdm

url = "https://ultravideo.fi/video/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"

# Ensure folder exists
import os
os.makedirs("/media/data/UVG", exist_ok=True)

# Stream download with progress bar
response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading Beauty")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)


# ----------------- Bosphorus -----------------
url = "https://ultravideo.fi/video/Bosphorus_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/UVG/Bosphorus_1920x1080_120fps_420_8bit_YUV_RAW.7z"

response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading Bosphorus")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)


# ----------------- HoneyBee -----------------
url = "https://ultravideo.fi/video/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/UVG/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z"

response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading HoneyBee")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)


✅ Download finished: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z


✅ Download finished: /media/data/UVG/Bosphorus_1920x1080_120fps_420_8bit_YUV_RAW.7z


✅ Download finished: /media/data/UVG/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z


In [10]:
# Install p7zip
!sudo apt-get install -y p7zip-full

# ----------------- Extract Beauty -----------------
!7z x /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/UVG/

# ----------------- Extract Bosphorus -----------------
!7z x /media/data/UVG/Bosphorus_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/UVG/

# ----------------- Extract HoneyBee -----------------
!7z x /media/data/UVG/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/UVG/

# ----------------- List extracted contents -----------------
!ls -lh /media/data/UVG


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /media/data/UVG/                          1 file, 925430047 bytes (883 MiB)

Extracting archive: /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
--
Path = /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
Type = 7z
Physical Size = 925430047
Headers Size = 246
Method = LZMA:16 BZip2
Solid = -
Blocks = 2

  0%    
Would you like to replace the existing file:
  Path:     /media/data/UVG/Beauty_1920x1080_120fps_420_8bit_YUV.yuv
  Size:     1866240000 bytes (1

In [11]:
!ls -lh /media/data/UVG


total 7.6G
-rw-r--r-- 1 root root 883M Sep  2 16:35 Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Beauty_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 Beauty_Copyright.txt
-rw-r--r-- 1 root root 650M Sep  2 16:35 Bosphorus_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Bosphorus_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 Bosphorus_Copyright.txt
-rw-r--r-- 1 root root 865M Sep  2 16:36 HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 HoneyBee_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 HoneyBee_Copyright.txt


In [13]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 4 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --save_decoded_frame 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1

/content/DCVC
testing 4 rates, using qp: 0, 21, 42, 63, 
  0% 0/212 [00:00<?, ?it/s]encoding/decoding 600 frames, average encoding time 45.078 ms, average decoding time 52.286 ms.
  0% 1/212 [02:53<10:09:33, 173.33s/it]encoding/decoding 600 frames, average encoding time 45.559 ms, average decoding time 52.745 ms.
  1% 2/212 [05:35<9:44:12, 166.92s/it] encoding/decoding 600 frames, average encoding time 45.581 ms, average decoding time 52.603 ms.
  1% 3/212 [08:17<9:33:45, 164.71s/it]encoding/decoding 600 frames, average encoding time 45.626 ms, average decoding time 53.034 ms.
  2% 4/212 [11:01<9:28:53, 164.10s/it]encoding/decoding 600 frames, average encoding time 45.759 ms, average decoding time 55.650 ms.
  2% 5/212 [13:58<9:42:29, 168.84s/it]encoding/decoding 600 frames, average encoding time 45.565 ms, average decoding time 53.209 ms.
  3% 6/212 [16:39<9:30:52, 166.28s/it]encoding/decoding 600 frames, average encoding time 45.588 ms, average decoding time 52.624 ms.
  3% 7/212 [19